## Reading phototransistor with a labJack T7\
Uses 1 analog input (AINs) to read the data.

Craig Lage - Jan 24, 2023

In [ ]:
import sys
import time, datetime
import numpy as np
from astropy.time import Time, TimeDelta

from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
# Open LabJack T4
handle = ljm.openS("T4", "ethernet", "139.229.170.19") 

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

deviceType = info[0]
print(deviceType)

In [ ]:
# Setup and call eWriteNames to configure AIN0 and AIN1 on the LabJack.
# LabJack T7 and other devices configuration

# AIN0 and AIN1:
#   Negative channel = single ended (199)
#   Range: +/-10.0 V (10.0)
#   Resolution index = Default (0)
#   Settling, in microseconds = Auto (0)
names = ["AIN0_NEGATIVE_CH", "AIN0_RANGE", "AIN0_RESOLUTION_INDEX", "AIN0_SETTLING_US",
         "AIN1_NEGATIVE_CH", "AIN1_RANGE", "AIN1_RESOLUTION_INDEX", "AIN1_SETTLING_US",
        "AIN2_NEGATIVE_CH", "AIN2_RANGE", "AIN2_RESOLUTION_INDEX", "AIN2_SETTLING_US"]
aValues = [199, 10.0, 0, 0,
           199, 10.0, 0, 0,
            199, 10.0, 0, 0]
numFrames = len(names)
ljm.eWriteNames(handle, numFrames, names, aValues)

In [ ]:
print("\nSet configuration:")
for i in range(numFrames):
    print("    %s : %f" % (names[i], aValues[i]))


In [ ]:
# Read AIN0
def readPhotodiode():
    loopAmount = 10
    # Read AIN's from the LabJack with eReadNames in a loop.
    names = ["AIN0"]
    numFrames = 1
    intervalHandle = 1
    ljm.startInterval(intervalHandle, 50000)  # Delay between readings (in microseconds)
    Results = []
    start = time.time()
    for n in range(loopAmount):
        results = ljm.eReadNames(handle, numFrames, names)
        Results.append(results[0])
        ljm.waitForNextInterval(intervalHandle)
        stamp = time.time()
        delay = stamp - start
        start = stamp

    return np.mean(Results)

In [ ]:
readPhotodiode()

In [ ]:
ljm.close(handle)